# 🚀 Multi-Input Hybrid Deep Learning Crypto Trading System
## Complete Technical Documentation & Testing Notebook

---

### Table of Contents

1. [System Overview & Architecture](#1-system-overview)
2. [Environment Setup](#2-environment-setup)
3. [Data Acquisition Pipeline](#3-data-acquisition)
4. [Feature Engineering](#4-feature-engineering)
5. [Technical Indicators](#5-technical-indicators)
6. [Model Architectures](#6-model-architectures)
7. [Single Asset Training & Prediction](#7-single-asset-training)
8. [Multi-Asset Portfolio Optimization](#8-portfolio-optimization)
9. [Correlation & Risk Analysis](#9-correlation-analysis)
10. [Deep Learning Portfolio Strategy](#10-deep-learning-portfolio)
11. [Reinforcement Learning Strategy](#11-rl-strategy)
12. [Hybrid Ensemble Strategy](#12-hybrid-strategy)
13. [Model Persistence](#13-model-persistence)
14. [Backtesting](#14-backtesting)
15. [AI Advisor Integration](#15-ai-advisor)
16. [Complete Workflow Demo](#16-complete-workflow)

---

**Author:** Crypto Trading System  
**Version:** 2.0  
**Last Updated:** February 2026

---
# 1. System Overview & Architecture <a id='1-system-overview'></a>

## System Goals

1. **Price Prediction** - Predict whether asset price will go UP or DOWN
2. **Portfolio Allocation** - Determine optimal capital allocation across multiple assets
3. **Risk Management** - Minimize losses while maximizing returns

## Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    FRONTEND (React.js)                          │
│  Dashboard │ Training │ Portfolio │ Backtesting │ AI Advisor    │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    BACKEND (FastAPI)                            │
├─────────────────────────────────────────────────────────────────┤
│  Data Pipeline    │  Training Service  │  Portfolio Optimizer   │
│  - CCXT Exchange  │  - 11+ Models      │  - Traditional+ML      │
│  - Multi-Asset    │  - Walk-Forward    │  - Deep Learning       │
│  - Any Date Range │  - Hyperparameter  │  - RL Agent (PPO)      │
│                   │                    │  - Hybrid Ensemble     │
├─────────────────────────────────────────────────────────────────┤
│  ML Models (TensorFlow/Keras)          │  LLM Service           │
│  - LSTM, GRU, Transformer              │  - OpenAI GPT          │
│  - CNN-LSTM, Attention                 │  - Claude              │
│  - TCN-GNN-LSTM Hybrid                 │  - Gemini              │
│  - RL (DQN, PPO)                       │                        │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    DATA SOURCES                                  │
│  Binance │ OKX │ KuCoin │ Bybit │ Kraken │ Coinbase             │
└─────────────────────────────────────────────────────────────────┘
```

---
# 2. Environment Setup <a id='2-environment-setup'></a>

In [ ]:
# Import required libraries
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
sns.set_palette("husl")

# API Configuration
API_BASE_URL = "https://work-1-oqzcuohwva-ue.a.run.app/api"

# Helper functions
def api_get(endpoint, params=None):
    """Make GET request to API"""
    try:
        response = requests.get(f"{API_BASE_URL}{endpoint}", params=params, timeout=60)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

def api_post(endpoint, data):
    """Make POST request to API"""
    try:
        response = requests.post(f"{API_BASE_URL}{endpoint}", json=data, timeout=120)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

def print_header(title):
    """Print formatted header"""
    print("\n" + "="*70)
    print(f"  {title}")
    print("="*70)

def print_success(msg):
    print(f"✅ {msg}")

def print_info(msg):
    print(f"ℹ️  {msg}")

def print_warning(msg):
    print(f"⚠️  {msg}")

print_header("ENVIRONMENT SETUP COMPLETE")
print_success(f"API URL: {API_BASE_URL}")
print_success("All libraries imported successfully")

In [ ]:
# Verify API connection
print_header("API CONNECTION TEST")

# Test endpoint
health = api_get("/dashboard/stats")

if "error" not in health:
    print_success("API connection successful!")
    print(f"\n📊 System Statistics:")
    print(f"   • Has Advanced Model: {health.get('has_advanced_model', False)}")
    print(f"   • Total Predictions Made: {health.get('total_predictions', 0)}")
    print(f"   • Total Training Runs: {health.get('total_training_runs', 0)}")
else:
    print_warning(f"API connection failed: {health.get('error')}")

---
# 3. Data Acquisition Pipeline <a id='3-data-acquisition'></a>

## Supported Exchanges
- **Binance** (default) - Largest crypto exchange
- **OKX** - Major derivatives exchange
- **KuCoin, Bybit, Kraken, Coinbase** - Additional support

## Data Types
- **OHLCV** - Open, High, Low, Close, Volume
- **Timeframes** - 1m, 5m, 15m, 1h, 4h, 1d, 1w

In [ ]:
print_header("DATA ACQUISITION - SINGLE ASSET")

# Fetch BTC/USDT data
symbol = "BTC/USDT"
timeframe = "1h"
limit = 500

print(f"\n📈 Fetching {symbol} data...")
print(f"   Timeframe: {timeframe}")
print(f"   Limit: {limit} candles")

market_data = api_get(f"/market-data?symbol={symbol}&timeframe={timeframe}&limit={limit}")

if market_data.get('data'):
    df = pd.DataFrame(market_data['data'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    
    print_success(f"Fetched {len(df)} candles")
    print(f"\n📅 Date Range:")
    print(f"   From: {df.index.min()}")
    print(f"   To: {df.index.max()}")
    
    print(f"\n📊 Price Statistics:")
    print(f"   Current Price: ${df['close'].iloc[-1]:,.2f}")
    print(f"   High: ${df['high'].max():,.2f}")
    print(f"   Low: ${df['low'].min():,.2f}")
    print(f"   Avg Volume: {df['volume'].mean():,.2f}")
    
    print(f"\n🔢 Sample Data (Last 5 candles):")
    display(df[['open', 'high', 'low', 'close', 'volume']].tail())
else:
    print_warning("Failed to fetch data")

In [ ]:
# Visualize price data
print_header("PRICE CHART VISUALIZATION")

fig, axes = plt.subplots(2, 1, figsize=(14, 8), gridspec_kw={'height_ratios': [3, 1]})

# Price chart
ax1 = axes[0]
ax1.plot(df.index, df['close'], color='#00d4aa', linewidth=1.5, label='Close Price')
ax1.fill_between(df.index, df['low'], df['high'], alpha=0.2, color='#00d4aa')
ax1.set_title(f'{symbol} Price Chart ({timeframe})', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price (USDT)', fontsize=12)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Volume chart
ax2 = axes[1]
colors = ['#00d4aa' if df['close'].iloc[i] >= df['open'].iloc[i] else '#ef4444' 
          for i in range(len(df))]
ax2.bar(df.index, df['volume'], color=colors, alpha=0.7, width=0.03)
ax2.set_ylabel('Volume', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print_success("Price chart generated")

In [ ]:
print_header("ARBITRARY DATE RANGE FETCHING")

print("\n📅 Testing long historical data fetch...")
print("   This feature allows fetching data from ANY date range (e.g., 2017 to today)")

# Preview data for 3-year range
data_preview = api_post("/training/data-preview", {
    "symbol": "BTC/USDT",
    "timeframe": "1d",
    "start_date": "2022-01-01T00:00:00Z",
    "end_date": "2024-12-31T00:00:00Z"
})

print(f"\n🔍 Data Preview (3-year daily data):")
print(f"   Symbol: {data_preview.get('symbol')}")
print(f"   Timeframe: {data_preview.get('timeframe')}")
print(f"   Start Date: {data_preview.get('start_date')}")
print(f"   End Date: {data_preview.get('end_date')}")
print(f"   Estimated Candles: {data_preview.get('estimated_candles'):,}")
print(f"   Estimated Training Samples: {data_preview.get('estimated_training_samples'):,}")
print(f"   Exchange: {data_preview.get('exchange')}")

if data_preview.get('size_warning'):
    print_warning(f"   {data_preview.get('size_warning')}")

---
# 4. Feature Engineering <a id='4-feature-engineering'></a>

## Feature Categories

1. **Price-based Features** - Returns, momentum, price ratios
2. **Volume Features** - Volume changes, volume ratios
3. **Technical Indicators** - RSI, MACD, Bollinger Bands, etc.
4. **Volatility Features** - ATR, historical volatility
5. **Market Regime** - Trend detection, regime classification

In [ ]:
print_header("FEATURE ENGINEERING DEMONSTRATION")

# Calculate features locally to demonstrate
feature_df = df.copy()

print("\n📊 Calculating Features...\n")

# 1. Returns
feature_df['return_1'] = feature_df['close'].pct_change(1)
feature_df['return_5'] = feature_df['close'].pct_change(5)
feature_df['return_10'] = feature_df['close'].pct_change(10)
feature_df['return_20'] = feature_df['close'].pct_change(20)
print_success("Returns (1, 5, 10, 20 periods)")

# 2. Moving Averages
feature_df['sma_20'] = feature_df['close'].rolling(20).mean()
feature_df['sma_50'] = feature_df['close'].rolling(50).mean()
feature_df['price_sma20_ratio'] = feature_df['close'] / feature_df['sma_20']
feature_df['price_sma50_ratio'] = feature_df['close'] / feature_df['sma_50']
print_success("Moving Averages (SMA 20, 50)")

# 3. RSI
delta = feature_df['close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
rs = gain / loss
feature_df['rsi'] = 100 - (100 / (1 + rs))
print_success("RSI (14 period)")

# 4. MACD
ema_12 = feature_df['close'].ewm(span=12).mean()
ema_26 = feature_df['close'].ewm(span=26).mean()
feature_df['macd'] = ema_12 - ema_26
feature_df['macd_signal'] = feature_df['macd'].ewm(span=9).mean()
feature_df['macd_hist'] = feature_df['macd'] - feature_df['macd_signal']
print_success("MACD (12, 26, 9)")

# 5. Bollinger Bands
feature_df['bb_middle'] = feature_df['close'].rolling(20).mean()
bb_std = feature_df['close'].rolling(20).std()
feature_df['bb_upper'] = feature_df['bb_middle'] + (bb_std * 2)
feature_df['bb_lower'] = feature_df['bb_middle'] - (bb_std * 2)
feature_df['bb_width'] = (feature_df['bb_upper'] - feature_df['bb_lower']) / feature_df['bb_middle']
feature_df['bb_percent'] = (feature_df['close'] - feature_df['bb_lower']) / (feature_df['bb_upper'] - feature_df['bb_lower'])
print_success("Bollinger Bands (20, 2)")

# 6. Volatility
feature_df['volatility_10'] = feature_df['return_1'].rolling(10).std()
feature_df['volatility_20'] = feature_df['return_1'].rolling(20).std()
print_success("Volatility (10, 20 period)")

# 7. Volume Features
feature_df['volume_sma'] = feature_df['volume'].rolling(20).mean()
feature_df['volume_ratio'] = feature_df['volume'] / feature_df['volume_sma']
print_success("Volume Features")

# 8. ATR (Average True Range)
high_low = feature_df['high'] - feature_df['low']
high_close = abs(feature_df['high'] - feature_df['close'].shift())
low_close = abs(feature_df['low'] - feature_df['close'].shift())
tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
feature_df['atr'] = tr.rolling(14).mean()
feature_df['atr_percent'] = feature_df['atr'] / feature_df['close']
print_success("ATR (14 period)")

print(f"\n📋 Total Features Created: {len([c for c in feature_df.columns if c not in df.columns])}")

In [ ]:
# Display feature statistics
print_header("FEATURE STATISTICS")

feature_cols = ['return_1', 'return_5', 'rsi', 'macd', 'bb_percent', 
                'volatility_10', 'volume_ratio', 'atr_percent']

print("\n📊 Feature Summary Statistics:")
display(feature_df[feature_cols].describe().round(4))

In [ ]:
# Visualize key features
print_header("FEATURE VISUALIZATION")

fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# RSI
ax1 = axes[0]
ax1.plot(feature_df.index[-100:], feature_df['rsi'].iloc[-100:], color='#00d4aa', linewidth=1.5)
ax1.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
ax1.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
ax1.set_title('RSI (Relative Strength Index)', fontsize=12, fontweight='bold')
ax1.set_ylabel('RSI')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# MACD
ax2 = axes[1]
ax2.plot(feature_df.index[-100:], feature_df['macd'].iloc[-100:], color='#00d4aa', linewidth=1.5, label='MACD')
ax2.plot(feature_df.index[-100:], feature_df['macd_signal'].iloc[-100:], color='#f59e0b', linewidth=1.5, label='Signal')
ax2.bar(feature_df.index[-100:], feature_df['macd_hist'].iloc[-100:], color='#8b5cf6', alpha=0.5, label='Histogram')
ax2.set_title('MACD (Moving Average Convergence Divergence)', fontsize=12, fontweight='bold')
ax2.set_ylabel('MACD')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Bollinger Bands %
ax3 = axes[2]
ax3.plot(feature_df.index[-100:], feature_df['bb_percent'].iloc[-100:], color='#00d4aa', linewidth=1.5)
ax3.axhline(y=1, color='red', linestyle='--', alpha=0.7)
ax3.axhline(y=0, color='green', linestyle='--', alpha=0.7)
ax3.set_title('Bollinger Band %B', fontsize=12, fontweight='bold')
ax3.set_ylabel('%B')
ax3.grid(True, alpha=0.3)

# Volatility
ax4 = axes[3]
ax4.plot(feature_df.index[-100:], feature_df['volatility_10'].iloc[-100:] * 100, color='#ef4444', linewidth=1.5, label='10-period')
ax4.plot(feature_df.index[-100:], feature_df['volatility_20'].iloc[-100:] * 100, color='#00d4aa', linewidth=1.5, label='20-period')
ax4.set_title('Historical Volatility', fontsize=12, fontweight='bold')
ax4.set_ylabel('Volatility (%)')
ax4.set_xlabel('Date')
ax4.legend(loc='upper right')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print_success("Feature visualization complete")

---
# 5. Technical Indicators <a id='5-technical-indicators'></a>

## Complete List of Indicators Used

| Category | Indicator | Description |
|----------|-----------|-------------|
| Trend | SMA (20, 50) | Simple Moving Average |
| Trend | EMA (12, 26) | Exponential Moving Average |
| Momentum | RSI (6, 14, 24) | Relative Strength Index |
| Momentum | MACD | Moving Average Convergence Divergence |
| Momentum | Stochastic | %K and %D oscillators |
| Momentum | ROC | Rate of Change |
| Volatility | Bollinger Bands | Price bands with std dev |
| Volatility | ATR | Average True Range |
| Volatility | Historical Vol | Rolling standard deviation |
| Volume | Volume Ratio | Volume vs Moving Average |
| Volume | MFI | Money Flow Index |
| Trend Strength | ADX | Average Directional Index |
| Trend Strength | DI+/DI- | Directional Indicators |

In [ ]:
print_header("COMPLETE TECHNICAL INDICATOR LIST")

# Get data info from training endpoint to see all features
data_info = api_post("/training/data-info", {
    "symbol": "BTC/USDT",
    "timeframe": "1h",
    "start_date": None,
    "end_date": None
})

if data_info.get('feature_names'):
    features = data_info['feature_names']
    print(f"\n📊 Total Features Available: {len(features)}")
    print(f"\n🔢 Feature List:")
    
    # Group features by category
    categories = {
        'Price Ratios': [f for f in features if 'ratio' in f or 'price' in f],
        'RSI': [f for f in features if 'rsi' in f],
        'MACD': [f for f in features if 'macd' in f],
        'Bollinger': [f for f in features if 'bb' in f],
        'Volatility': [f for f in features if 'volatil' in f or 'atr' in f],
        'Volume': [f for f in features if 'volume' in f or 'mfi' in f],
        'Trend': [f for f in features if 'adx' in f or 'di_' in f],
        'Returns': [f for f in features if 'return' in f],
        'Stochastic': [f for f in features if 'stoch' in f],
        'Other': [f for f in features if not any(x in f for x in ['ratio', 'rsi', 'macd', 'bb', 'volatil', 'atr', 'volume', 'mfi', 'adx', 'di_', 'return', 'stoch', 'price'])]
    }
    
    for cat, feats in categories.items():
        if feats:
            print(f"\n   {cat}:")
            for f in feats:
                print(f"      • {f}")
else:
    print_info("Feature list not available - showing calculated features")
    print(f"\nCalculated features: {list(feature_df.columns)}")

---
# 6. Model Architectures <a id='6-model-architectures'></a>

## Available Neural Network Architectures

### 1. Recurrent Neural Networks
- **LSTM** - Long Short-Term Memory
- **GRU** - Gated Recurrent Unit
- **Bi-LSTM** - Bidirectional LSTM

### 2. Attention-Based Models
- **Transformer** - Self-attention mechanism
- **Attention GRU** - GRU with attention layer

### 3. Hybrid Models
- **CNN-LSTM** - Convolutional + LSTM
- **TCN-GNN-LSTM** - Temporal Conv + Graph NN + LSTM

### 4. Reinforcement Learning
- **DQN** - Deep Q-Network
- **PPO** - Proximal Policy Optimization

### 5. Ensemble
- **Multi-Model Ensemble** - Voting, Stacking, Bagging

In [ ]:
print_header("MODEL ARCHITECTURES")

models = [
    {
        "name": "LSTM",
        "type": "Recurrent",
        "description": "Long Short-Term Memory - Captures long-term dependencies in sequences",
        "best_for": "Time series with long-term patterns",
        "params": "~50K-200K"
    },
    {
        "name": "GRU",
        "type": "Recurrent",
        "description": "Gated Recurrent Unit - Simpler than LSTM, faster training",
        "best_for": "When training speed is important",
        "params": "~40K-150K"
    },
    {
        "name": "Bi-LSTM",
        "type": "Recurrent",
        "description": "Processes sequence in both forward and backward directions",
        "best_for": "When context from both directions matters",
        "params": "~100K-400K"
    },
    {
        "name": "Transformer",
        "type": "Attention",
        "description": "Self-attention mechanism for parallel processing",
        "best_for": "Long sequences, capturing global dependencies",
        "params": "~200K-1M"
    },
    {
        "name": "CNN-LSTM",
        "type": "Hybrid",
        "description": "CNN extracts features, LSTM learns temporal patterns",
        "best_for": "Feature extraction + sequence learning",
        "params": "~80K-300K"
    },
    {
        "name": "TCN-GNN-LSTM",
        "type": "Advanced Hybrid",
        "description": "Temporal Conv + Graph Neural Network + LSTM",
        "best_for": "Multi-scale patterns with asset relationships",
        "params": "~300K-800K"
    },
    {
        "name": "DQN (RL)",
        "type": "Reinforcement Learning",
        "description": "Deep Q-Network - Learns optimal trading actions",
        "best_for": "Direct trading signal generation",
        "params": "~100K-500K"
    },
    {
        "name": "PPO (RL)",
        "type": "Reinforcement Learning",
        "description": "Proximal Policy Optimization - Stable policy learning",
        "best_for": "Continuous action spaces, portfolio allocation",
        "params": "~150K-600K"
    }
]

print("\n🧠 Available Model Architectures:\n")
for i, model in enumerate(models, 1):
    print(f"{'='*60}")
    print(f"{i}. {model['name']} ({model['type']})")
    print(f"{'='*60}")
    print(f"   📝 Description: {model['description']}")
    print(f"   🎯 Best For: {model['best_for']}")
    print(f"   📊 Parameters: {model['params']}")
    print()

---
# 7. Single Asset Training & Prediction <a id='7-single-asset-training'></a>

In [ ]:
print_header("TRAINING STATUS")

# Check current training status
training_status = api_get("/training/advanced/status")

print(f"\n📊 Current Training Status:")
print(f"   Is Training: {training_status.get('is_training', False)}")
print(f"   Current Model: {training_status.get('current_model', 'N/A')}")

if training_status.get('final_accuracy'):
    print(f"\n✅ Last Training Results:")
    print(f"   Model Type: {training_status.get('model_type', 'N/A')}")
    print(f"   Final Accuracy: {training_status.get('final_accuracy', 0)*100:.2f}%")
    print(f"   Total Epochs: {training_status.get('total_epochs', 0)}")
    print(f"   Current Epoch: {training_status.get('current_epoch', 0)}")

if training_status.get('data_info'):
    data_info = training_status['data_info']
    print(f"\n📈 Training Data Info:")
    print(f"   Samples: {data_info.get('samples', 0):,}")
    print(f"   Features: {data_info.get('features', 0)}")
    print(f"   Class Balance: {data_info.get('class_balance', {})}")

In [ ]:
print_header("PRICE PREDICTION")

# Make prediction
print("\n🔮 Making price prediction for BTC/USDT...")

prediction = api_get("/predictions/make?symbol=BTC/USDT&use_advanced=true")

if prediction.get('direction_label'):
    print(f"\n{'='*50}")
    print(f"          PREDICTION RESULT")
    print(f"{'='*50}")
    print(f"\n   Symbol: {prediction.get('symbol')}")
    print(f"   Current Price: ${prediction.get('current_price', 0):,.2f}")
    print(f"\n   🎯 PREDICTION: {prediction.get('direction_label')}")
    print(f"   📊 Confidence: {prediction.get('confidence', 0)*100:.1f}%")
    print(f"   🧠 Model: {prediction.get('model_type', 'N/A')}")
    print(f"   ⏰ Timestamp: {prediction.get('timestamp', 'N/A')}")
    
    # Visualize confidence
    confidence = prediction.get('confidence', 0)
    direction = prediction.get('direction_label', 'HOLD')
    
    fig, ax = plt.subplots(figsize=(8, 2))
    color = '#00d4aa' if direction == 'UP' else '#ef4444' if direction == 'DOWN' else '#f59e0b'
    ax.barh([0], [confidence * 100], color=color, height=0.5)
    ax.set_xlim(0, 100)
    ax.set_yticks([])
    ax.set_xlabel('Confidence (%)')
    ax.set_title(f'Prediction: {direction} ({confidence*100:.1f}%)', fontsize=12, fontweight='bold')
    ax.axvline(x=50, color='white', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()
else:
    print_warning("Prediction not available - model may need training")

---
# 8. Multi-Asset Portfolio Optimization <a id='8-portfolio-optimization'></a>

## Portfolio Optimization Strategies

1. **Traditional + ML** - Mean-Variance Optimization with ML-predicted returns
2. **Deep Learning** - Neural network directly outputs allocation weights
3. **RL Agent (PPO)** - Reinforcement learning for dynamic rebalancing
4. **Hybrid Ensemble** - Combines all strategies

## Optimization Objectives
- **Max Sharpe Ratio** - Best risk-adjusted return
- **Max Return** - Aggressive, highest expected return
- **Min Risk** - Conservative, lowest volatility
- **Risk Parity** - Equal risk contribution from each asset

In [ ]:
print_header("PORTFOLIO OPTIMIZATION - SETUP")

# Get available assets
assets_info = api_get("/portfolio/assets")

print(f"\n📊 Portfolio Configuration:")
print(f"\n   Available Assets ({assets_info.get('count', 0)}):")
for i, asset in enumerate(assets_info.get('assets', []), 1):
    print(f"      {i:2}. {asset}")

print(f"\n   Strategies: {assets_info.get('strategies')}")
print(f"   Objectives: {assets_info.get('objectives')}")
print(f"   Horizons: {assets_info.get('horizons')}")

In [ ]:
print_header("FETCHING MULTI-ASSET DATA")

# Select assets for portfolio
selected_assets = [
    "BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT",
    "ADA/USDT", "DOGE/USDT", "AVAX/USDT", "DOT/USDT", "MATIC/USDT"
]

print(f"\n📈 Fetching data for {len(selected_assets)} assets...")
print(f"   Assets: {[a.replace('/USDT', '') for a in selected_assets]}")

fetch_result = api_post("/portfolio/fetch-data", {
    "assets": selected_assets,
    "timeframe": "1d"
})

if fetch_result.get('status') == 'success':
    print_success(f"Fetched data for {fetch_result.get('assets_fetched')} assets")
    
    # Display asset statistics
    if fetch_result.get('statistics'):
        print(f"\n📊 Asset Statistics:")
        stats_df = pd.DataFrame(fetch_result['statistics']).T
        stats_df = stats_df[['current_price', 'expected_return', 'volatility', 'sharpe_ratio', 'data_points']]
        stats_df['current_price'] = stats_df['current_price'].apply(lambda x: f"${x:,.2f}")
        stats_df['expected_return'] = stats_df['expected_return'].apply(lambda x: f"{x:+.2f}%")
        stats_df['volatility'] = stats_df['volatility'].apply(lambda x: f"{x:.2f}%")
        display(stats_df)
else:
    print_warning(f"Fetch failed: {fetch_result}")

In [ ]:
print_header("PORTFOLIO OPTIMIZATION - ALL 4 STRATEGIES")

investment_amount = 10000

print(f"\n💰 Investment Amount: ${investment_amount:,}")
print(f"📊 Objective: Maximize Sharpe Ratio")
print(f"⏱️  Horizon: 7 days")
print(f"\n🔄 Running optimization with all 4 strategies...")

optimization_result = api_post("/portfolio/optimize", {
    "assets": selected_assets,
    "investment_amount": investment_amount,
    "strategy": "traditional_ml",
    "objective": "max_sharpe",
    "horizon": "7d",
    "compare_all": True,
    "constraints": {
        "max_weight": 25,
        "min_assets": 5
    }
})

if optimization_result.get('status') == 'success':
    print_success("Optimization complete!")
    print(f"\n⭐ Recommended Strategy: {optimization_result.get('recommended', 'N/A').upper()}")
    
    print(f"\n{'='*70}")
    print("                    STRATEGY COMPARISON")
    print(f"{'='*70}")
    
    strategy_results = []
    for strategy_name, strategy_data in optimization_result.get('strategies', {}).items():
        status = strategy_data.get('status')
        if status == 'success':
            metrics = strategy_data.get('metrics', {})
            strategy_results.append({
                'Strategy': strategy_name,
                'Expected Return': f"{metrics.get('expected_return', 0):+.2f}%",
                'Volatility': f"{metrics.get('volatility', 0):.2f}%",
                'Sharpe Ratio': f"{metrics.get('sharpe_ratio', 0):.3f}",
                'Assets Used': metrics.get('num_assets', 0),
                'Status': '✅'
            })
        else:
            strategy_results.append({
                'Strategy': strategy_name,
                'Expected Return': '-',
                'Volatility': '-',
                'Sharpe Ratio': '-',
                'Assets Used': '-',
                'Status': '⏳ Not Trained' if status == 'not_trained' else '❌'
            })
    
    results_df = pd.DataFrame(strategy_results)
    display(results_df)
else:
    print_warning(f"Optimization failed: {optimization_result}")

In [ ]:
print_header("RECOMMENDED ALLOCATION")

recommended = optimization_result.get('recommended', 'traditional_ml')
strategy_data = optimization_result.get('strategies', {}).get(recommended, {})

if strategy_data.get('allocations'):
    allocations = strategy_data['allocations']
    alloc_df = pd.DataFrame(allocations)
    
    print(f"\n💼 Strategy: {recommended.upper()}")
    print(f"💰 Total Investment: ${investment_amount:,}")
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Pie chart
    ax1 = axes[0]
    colors = plt.cm.Set3(np.linspace(0, 1, len(alloc_df)))
    wedges, texts, autotexts = ax1.pie(
        alloc_df['weight'], 
        labels=[a.replace('/USDT', '') for a in alloc_df['symbol']], 
        autopct='%1.1f%%',
        colors=colors,
        explode=[0.02] * len(alloc_df)
    )
    ax1.set_title('Portfolio Allocation', fontsize=14, fontweight='bold')
    
    # Bar chart
    ax2 = axes[1]
    symbols = [a.replace('/USDT', '') for a in alloc_df['symbol']]
    amounts = alloc_df['amount']
    bars = ax2.barh(symbols, amounts, color='#00d4aa')
    ax2.set_xlabel('Amount ($)', fontsize=12)
    ax2.set_title('Investment per Asset', fontsize=14, fontweight='bold')
    
    # Add value labels
    for bar, amount in zip(bars, amounts):
        ax2.text(amount + 50, bar.get_y() + bar.get_height()/2, 
                f'${amount:,.0f}', va='center', fontsize=10)
    
    ax2.set_xlim(0, max(amounts) * 1.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print allocation table
    print(f"\n📋 Allocation Details:")
    alloc_df['Amount'] = alloc_df['amount'].apply(lambda x: f"${x:,.2f}")
    alloc_df['Weight'] = alloc_df['weight'].apply(lambda x: f"{x:.1f}%")
    alloc_df['Expected Return'] = alloc_df['expected_return'].apply(lambda x: f"{x:+.2f}%")
    display(alloc_df[['symbol', 'Weight', 'Amount', 'Expected Return']])
    
    # Portfolio metrics
    metrics = strategy_data.get('metrics', {})
    print(f"\n📊 Portfolio Metrics:")
    print(f"   Expected Return: {metrics.get('expected_return', 0):+.2f}%")
    print(f"   Volatility (Risk): {metrics.get('volatility', 0):.2f}%")
    print(f"   Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
    print(f"   Assets in Portfolio: {metrics.get('num_assets', 0)}")

---
# 9. Correlation & Risk Analysis <a id='9-correlation-analysis'></a>

In [ ]:
print_header("CORRELATION MATRIX")

correlation = api_get("/portfolio/correlation")

if correlation.get('status') == 'success':
    corr_data = correlation.get('data', {})
    assets = corr_data.get('assets', [])
    matrix = np.array(corr_data.get('matrix', []))
    
    # Create correlation heatmap
    fig, ax = plt.subplots(figsize=(12, 10))
    
    mask = np.triu(np.ones_like(matrix, dtype=bool), k=1)
    cmap = sns.diverging_palette(250, 10, as_cmap=True)
    
    sns.heatmap(
        matrix, 
        mask=mask,
        annot=True, 
        fmt='.2f', 
        cmap=cmap,
        center=0,
        square=True,
        linewidths=0.5,
        xticklabels=assets,
        yticklabels=assets,
        cbar_kws={'shrink': 0.8},
        ax=ax
    )
    
    ax.set_title('Asset Correlation Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Best diversification pairs
    div_pairs = correlation.get('diversification_pairs', [])
    if div_pairs:
        print(f"\n🎯 Best Diversification Pairs (Low Correlation):")
        for i, pair in enumerate(div_pairs[:10], 1):
            print(f"   {i:2}. {pair[0]:12} ↔ {pair[1]:12} : {pair[2]:+.3f}")
else:
    print_warning("Correlation data not available")

In [ ]:
print_header("EFFICIENT FRONTIER")

frontier = api_get(f"/portfolio/efficient-frontier?assets={','.join(selected_assets)}")

if frontier.get('status') == 'success':
    frontier_data = frontier.get('frontier', [])
    
    if frontier_data:
        frontier_df = pd.DataFrame(frontier_data)
        
        # Create efficient frontier plot
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Plot frontier
        scatter = ax.scatter(
            frontier_df['volatility'], 
            frontier_df['return'],
            c=frontier_df['sharpe'],
            cmap='viridis',
            s=100,
            alpha=0.7
        )
        
        # Connect points with line
        ax.plot(frontier_df['volatility'], frontier_df['return'], 
                color='#00d4aa', linewidth=2, alpha=0.5)
        
        # Mark optimal point (max Sharpe)
        optimal_idx = frontier_df['sharpe'].idxmax()
        optimal = frontier_df.loc[optimal_idx]
        ax.scatter([optimal['volatility']], [optimal['return']], 
                  color='red', s=300, marker='*', zorder=5, label='Optimal (Max Sharpe)')
        
        # Labels and formatting
        ax.set_xlabel('Risk (Volatility %)', fontsize=12)
        ax.set_ylabel('Expected Return (%)', fontsize=12)
        ax.set_title('Efficient Frontier - Risk vs Return', fontsize=14, fontweight='bold')
        ax.legend(loc='upper left')
        ax.grid(True, alpha=0.3)
        
        # Colorbar
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label('Sharpe Ratio', fontsize=12)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n⭐ Optimal Portfolio Point:")
        print(f"   Expected Return: {optimal['return']:.2f}%")
        print(f"   Volatility: {optimal['volatility']:.2f}%")
        print(f"   Sharpe Ratio: {optimal['sharpe']:.3f}")
else:
    print_warning("Efficient frontier data not available")

---
# 10. Deep Learning Portfolio Strategy <a id='10-deep-learning-portfolio'></a>

## Architecture
```
Input (Market Features) → LSTM Encoder → Attention Layer → Dense Layers → Softmax → Portfolio Weights
```

## Training Objective
- **Loss Function**: Negative Sharpe Ratio
- **Goal**: Maximize risk-adjusted returns

In [ ]:
print_header("DEEP LEARNING PORTFOLIO MODEL")

model_info = api_get("/portfolio/model-info")

print(f"\n🧠 Deep Learning Model Status:")
print(f"   Trained: {model_info.get('deep_learning_trained', False)}")

if model_info.get('deep_learning_info'):
    dl_info = model_info['deep_learning_info']
    print(f"\n📊 Model Details:")
    print(f"   Assets: {dl_info.get('n_assets', 0)}")
    print(f"   Lookback Window: {dl_info.get('lookback', 0)} periods")
    print(f"   Feature Dimension: {dl_info.get('feature_dim', 0)}")
    print(f"   Total Parameters: {dl_info.get('model_params', 0):,}")
    
    if dl_info.get('asset_names'):
        print(f"   Assets Trained On: {dl_info['asset_names']}")
else:
    print_info("Deep Learning model not trained yet")
    print("   To train: Use the Portfolio page or call /api/portfolio/train-model")

---
# 11. Reinforcement Learning Strategy <a id='11-rl-strategy'></a>

## PPO (Proximal Policy Optimization)

### Components
- **Actor Network**: Outputs portfolio weights (policy)
- **Critic Network**: Estimates value function

### Environment
- **State**: Asset features + current portfolio weights
- **Action**: New portfolio weights
- **Reward**: Portfolio return - transaction costs

In [ ]:
print_header("REINFORCEMENT LEARNING PORTFOLIO AGENT")

print(f"\n🤖 RL Agent Status:")
print(f"   Trained: {model_info.get('rl_agent_trained', False)}")

if model_info.get('rl_agent_info'):
    rl_info = model_info['rl_agent_info']
    print(f"\n📊 Agent Details:")
    print(f"   Assets: {rl_info.get('n_assets', 0)}")
    
    if rl_info.get('training_result'):
        tr = rl_info['training_result']
        print(f"\n📈 Training Results:")
        print(f"   Episodes: {tr.get('episodes', 0)}")
        print(f"   Final Portfolio Value: ${tr.get('final_portfolio_value', 0):,.2f}")
        print(f"   Total Return: {tr.get('total_return', 0):+.2f}%")
        print(f"   Final Avg Reward: {tr.get('final_avg_reward', 0):.4f}")
    
    if rl_info.get('asset_names'):
        print(f"   Assets: {rl_info['asset_names']}")
else:
    print_info("RL Agent not trained yet")
    print("   To train: Use the Portfolio page or call /api/portfolio/train-model")

---
# 12. Hybrid Ensemble Strategy <a id='12-hybrid-strategy'></a>

## How It Works

1. Get allocation from **Traditional+ML**
2. Get allocation from **Deep Learning** (if trained)
3. Get allocation from **RL Agent** (if trained)
4. **Combine** using weighted average
5. Apply **constraints** and normalize

In [ ]:
print_header("HYBRID ENSEMBLE STRATEGY")

# Check which strategies are available
strategies_available = [
    ("Traditional+ML", True),
    ("Deep Learning", model_info.get('deep_learning_trained', False)),
    ("RL Agent", model_info.get('rl_agent_trained', False))
]

print(f"\n📊 Strategies Available for Hybrid:")
for name, available in strategies_available:
    status = "✅ Available" if available else "❌ Not Trained"
    print(f"   • {name}: {status}")

# Get hybrid allocation if available
hybrid_data = optimization_result.get('strategies', {}).get('hybrid', {})

if hybrid_data.get('status') == 'success':
    print(f"\n🔀 Hybrid Allocation Results:")
    metrics = hybrid_data.get('metrics', {})
    print(f"   Expected Return: {metrics.get('expected_return', 0):+.2f}%")
    print(f"   Volatility: {metrics.get('volatility', 0):.2f}%")
    print(f"   Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
    
    if hybrid_data.get('strategies_combined'):
        print(f"\n   Strategies Combined: {hybrid_data['strategies_combined']}")

---
# 13. Model Persistence <a id='13-model-persistence'></a>

## Features
- **Save** trained models to disk
- **Load** models without retraining
- **Delete** unused models
- **List** all saved models

In [ ]:
print_header("SAVED PORTFOLIO MODELS")

saved_models = api_get("/portfolio/models/list")

print(f"\n💾 Total Saved Models: {saved_models.get('total_models', 0)}")

# Deep Learning Models
dl_models = saved_models.get('deep_learning_models', [])
print(f"\n🧠 Deep Learning Models ({len(dl_models)}):")
if dl_models:
    for model in dl_models:
        print(f"   ┌─ {model['model_name']}")
        print(f"   │  Assets: {model.get('n_assets', 0)}")
        print(f"   │  Parameters: {model.get('model_params', 0):,}")
        print(f"   └─ Created: {model.get('created_at', 'N/A')}")
        print()
else:
    print("   No saved DL models")

# RL Models
rl_models = saved_models.get('rl_agent_models', [])
print(f"\n🤖 RL Agent Models ({len(rl_models)}):")
if rl_models:
    for model in rl_models:
        print(f"   ┌─ {model['model_name']}")
        print(f"   │  Assets: {model.get('n_assets', 0)}")
        if model.get('training_result'):
            tr = model['training_result']
            print(f"   │  Training Return: {tr.get('total_return', 0):+.2f}%")
            print(f"   │  Episodes: {tr.get('episodes', 0)}")
        print(f"   └─ Created: {model.get('created_at', 'N/A')}")
        print()
else:
    print("   No saved RL models")

In [ ]:
# Demonstrate loading a saved model
print_header("MODEL LOAD DEMONSTRATION")

if dl_models:
    model_to_load = dl_models[0]
    print(f"\n📂 Loading model: {model_to_load['model_name']}")
    
    load_result = api_post("/portfolio/models/load", {
        "model_type": "deep_learning",
        "model_path": model_to_load['model_path']
    })
    
    if load_result.get('status') == 'success':
        print_success("Model loaded successfully!")
        print(f"   Assets: {load_result.get('n_assets', 0)}")
        print(f"   Asset Names: {load_result.get('asset_names', [])}")
    else:
        print_warning(f"Load failed: {load_result.get('message', 'Unknown error')}")
else:
    print_info("No saved models to demonstrate loading")

---
# 14. Backtesting <a id='14-backtesting'></a>

In [ ]:
print_header("BACKTESTING STATUS")

backtest_status = api_get("/backtest/status")

print(f"\n📊 Backtest System Status:")
print(f"   Is Running: {backtest_status.get('is_running', False)}")

# Get backtest history
backtest_history = api_get("/backtest/history?limit=5")

if backtest_history.get('history'):
    print(f"\n📜 Recent Backtests ({backtest_history.get('count', 0)}):")
    for bt in backtest_history['history']:
        print(f"   • {bt.get('created_at', 'N/A')[:19]}")
        if bt.get('metrics'):
            m = bt['metrics']
            print(f"     Total Return: {m.get('total_return', 0):+.2f}%")
            print(f"     Win Rate: {m.get('win_rate', 0):.1f}%")
else:
    print_info("No backtest history available")

---
# 15. AI Advisor Integration <a id='15-ai-advisor'></a>

## Supported LLMs
- **OpenAI GPT-4o**
- **Anthropic Claude**
- **Google Gemini**

In [ ]:
print_header("AI ADVISOR")

print("\n🤖 Testing AI Advisor...")
print("   Model: GPT-4o")
print("   Query: Market analysis request\n")

advisor_response = api_post("/llm/chat", {
    "message": "Give me a brief 2-sentence analysis of the current Bitcoin market conditions.",
    "model": "gpt-4o"
})

if advisor_response.get('response'):
    print(f"{'='*60}")
    print("AI ADVISOR RESPONSE")
    print(f"{'='*60}")
    print(f"\n{advisor_response['response']}")
    print(f"\n{'='*60}")
    print(f"Model: {advisor_response.get('model', 'N/A')}")
else:
    print_warning(f"AI Advisor error: {advisor_response.get('error', 'Unknown')}")

---
# 16. Complete Workflow Demo <a id='16-complete-workflow'></a>

In [ ]:
def complete_trading_workflow(investment_amount=10000):
    """
    Execute complete trading system workflow:
    Data → Analysis → Optimization → Recommendation
    """
    
    print("\n" + "="*70)
    print("         🚀 COMPLETE TRADING SYSTEM WORKFLOW 🚀")
    print("="*70)
    
    results = {}
    
    # Step 1: Fetch Data
    print("\n[STEP 1/5] 📈 Fetching Multi-Asset Data...")
    assets = ["BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT"]
    fetch = api_post("/portfolio/fetch-data", {"assets": assets, "timeframe": "1d"})
    print(f"           ✅ Fetched data for {fetch.get('assets_fetched', 0)} assets")
    results['data_fetched'] = fetch.get('assets_fetched', 0)
    
    # Step 2: Single Asset Prediction
    print("\n[STEP 2/5] 🔮 Making BTC Price Prediction...")
    prediction = api_get("/predictions/make?symbol=BTC/USDT&use_advanced=true")
    if prediction.get('direction_label'):
        print(f"           ✅ BTC Prediction: {prediction['direction_label']} ({prediction['confidence']*100:.1f}% confidence)")
        results['btc_prediction'] = prediction['direction_label']
        results['btc_confidence'] = prediction['confidence']
    
    # Step 3: Correlation Analysis
    print("\n[STEP 3/5] 🔗 Analyzing Asset Correlations...")
    corr = api_get("/portfolio/correlation")
    if corr.get('diversification_pairs'):
        best = corr['diversification_pairs'][0]
        print(f"           ✅ Best diversification: {best[0]} ↔ {best[1]} (corr: {best[2]})")
        results['best_diversification'] = f"{best[0]} ↔ {best[1]}"
    
    # Step 4: Portfolio Optimization
    print("\n[STEP 4/5] 🎯 Optimizing Portfolio Allocation...")
    opt = api_post("/portfolio/optimize", {
        "assets": assets,
        "investment_amount": investment_amount,
        "objective": "max_sharpe",
        "compare_all": True,
        "constraints": {"max_weight": 30, "min_assets": 3}
    })
    if opt.get('status') == 'success':
        print(f"           ✅ Recommended strategy: {opt['recommended']}")
        results['recommended_strategy'] = opt['recommended']
    
    # Step 5: Final Recommendation
    print("\n[STEP 5/5] 💼 Generating Final Recommendation...")
    
    recommended = opt.get('recommended', 'traditional_ml')
    strategy_data = opt.get('strategies', {}).get(recommended, {})
    
    print("\n" + "="*70)
    print("                    📊 FINAL RECOMMENDATION")
    print("="*70)
    
    if strategy_data.get('allocations'):
        print(f"\n💰 Investment: ${investment_amount:,}")
        print(f"📈 Strategy: {recommended.upper()}")
        print(f"\n┌{'─'*50}┐")
        print(f"│{'OPTIMAL PORTFOLIO ALLOCATION':^50}│")
        print(f"├{'─'*50}┤")
        
        for alloc in strategy_data['allocations']:
            symbol = alloc['symbol'].replace('/USDT', '')
            weight = alloc['weight']
            amount = alloc['amount']
            bar = '█' * int(weight / 2)
            print(f"│ {symbol:6} {bar:15} {weight:5.1f}% ${amount:>8,.0f} │")
        
        print(f"└{'─'*50}┘")
        
        metrics = strategy_data.get('metrics', {})
        print(f"\n📊 Expected Performance:")
        print(f"   • Expected Return: {metrics.get('expected_return', 0):+.2f}%")
        print(f"   • Risk (Volatility): {metrics.get('volatility', 0):.2f}%")
        print(f"   • Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
        
        results['expected_return'] = metrics.get('expected_return', 0)
        results['risk'] = metrics.get('volatility', 0)
        results['sharpe'] = metrics.get('sharpe_ratio', 0)
        results['allocations'] = strategy_data['allocations']
    
    print("\n" + "="*70)
    print("         ✅ WORKFLOW COMPLETE")
    print("="*70)
    
    return results

# Execute the complete workflow
workflow_results = complete_trading_workflow(investment_amount=10000)

---
# Summary

## System Capabilities Demonstrated

| Feature | Status | Description |
|---------|--------|-------------|
| Multi-Exchange Data | ✅ | Binance, OKX, and more |
| Arbitrary Date Range | ✅ | Fetch data from 2017 to today |
| Feature Engineering | ✅ | 30+ technical indicators |
| Price Prediction | ✅ | 11+ neural network architectures |
| Portfolio Optimization | ✅ | 4 AI strategies |
| Correlation Analysis | ✅ | Asset correlation heatmap |
| Efficient Frontier | ✅ | Risk-return optimization |
| Model Persistence | ✅ | Save/Load/Delete models |
| AI Advisor | ✅ | GPT-4o, Claude, Gemini |

## Key API Endpoints

| Endpoint | Method | Description |
|----------|--------|-------------|
| `/api/market-data` | GET | Fetch OHLCV data |
| `/api/predictions/make` | GET | Get price prediction |
| `/api/training/advanced/start` | POST | Start model training |
| `/api/portfolio/fetch-data` | POST | Fetch multi-asset data |
| `/api/portfolio/optimize` | POST | Get optimal allocation |
| `/api/portfolio/correlation` | GET | Correlation matrix |
| `/api/portfolio/models/list` | GET | List saved models |
| `/api/portfolio/models/save` | POST | Save trained model |
| `/api/portfolio/models/load` | POST | Load saved model |
| `/api/llm/chat` | POST | AI Advisor chat |

---

**End of Notebook**